# 4326 COGs to 4326 COG Mosaic

In [1]:
import adlfs
import glob

In [2]:
# use read only token
sas_token = "se=2023-12-03T08%3A42Z&sp=racwdl&sv=2022-11-02&sr=c&skoid=598b2582-4d1d-4c4e-9bb2-3ad571b791b5&sktid=f6b6dd5b-f02f-441a-99a0-162ac5060bd2&skt=2023-11-26T08%3A42%3A56Z&ske=2023-12-03T08%3A42%3A00Z&sks=b&skv=2022-11-02&sig=d0zp1ahDN96QRbySIfd5oZzuCnz7DXg3iRQJELJJZsY%3D"

In [3]:
years = ['2015_median','2016_median','2017_median','2018_median','2019_median','2020_median','2021_median','2022_median','2023_median']
resolution = 80

In [4]:
fs = adlfs.AzureBlobFileSystem(account_name="snowmelt", credential=sas_token, anon=True)

In [5]:
# for year in years:
#     f = open(f'vrts/4326/filenames/files_{year}.txt','w')
#     for fn in fs.glob(f'snowmelt/eric/global_4326/*/*{year}*'):
#         f.write(f'https://snowmelt.blob.core.windows.net/{fn}\n')
#     f.close()

In [6]:
# for year in years:
#     if (year == 'allyears_std') | (year == 'allyears_corr_strength'):
#         nodata = "nan"
#     else:
#         nodata = '-32768'
        
#     print(f'for year {year}, running gdalbuiltvrt...')
#     !gdalbuildvrt -input_file_list $f'vrts/4326/filenames/files_{year}.txt' $f'vrts/4326/vrt_{year}.vrt' -vrtnodata $nodata
    
#     print(f'for year {year}, running gdalwarp...')
#     !gdalwarp $f'vrts/4326/vrt_{year}.vrt' $f'tif/4326/tif_{year}.tif' -of COG
    
#     print(f'for year {year}, uploading mosaic')
#     fs.upload(f'tif/4326/tif_{year}.tif',f'/snowmelt/eric/mosaics/global_4326/runoff_onset_mosaic_{year}_{resolution}m_4326.tif',overwrite=True)
    
#     !rm $f'tif/4326/tif_{year}.tif'
    

In [7]:
fs.glob('/snowmelt/eric/mosaics/global_4326/*')

['snowmelt/eric/mosaics/global_4326/runoff_onset_mosaic_2015_median_80m_4326.tif',
 'snowmelt/eric/mosaics/global_4326/runoff_onset_mosaic_2016_median_80m_4326.tif',
 'snowmelt/eric/mosaics/global_4326/runoff_onset_mosaic_2017_median_80m_4326.tif',
 'snowmelt/eric/mosaics/global_4326/runoff_onset_mosaic_2018_median_80m_4326.tif',
 'snowmelt/eric/mosaics/global_4326/runoff_onset_mosaic_2019_median_80m_4326.tif',
 'snowmelt/eric/mosaics/global_4326/runoff_onset_mosaic_2020_median_80m_4326.tif',
 'snowmelt/eric/mosaics/global_4326/runoff_onset_mosaic_2021_median_80m_4326.tif',
 'snowmelt/eric/mosaics/global_4326/runoff_onset_mosaic_2022_median_80m_4326.tif',
 'snowmelt/eric/mosaics/global_4326/runoff_onset_mosaic_2023_median_80m_4326.tif']

In [1]:
# read in all?? huge cogs, let's see how dangerous this is

# import xarray as xr
# import rioxarray as rxr
# import re
# import dask_gateway
# import numpy as np
# import io

# cluster = dask_gateway.GatewayCluster()
# client = cluster.get_client()
# cluster.scale(50)
# print(cluster.dashboard_link)

# url = "https://snowmelt.blob.core.windows.net/"
# year_list = []
# for geotiff in fs.glob('/snowmelt/eric/mosaics/global_4326/*'):
#     year_list.append(re.findall("_(\d{4})_", geotiff)[0])
# year_list = [int(year) for year in year_list]

# #Create variable used for time axis
# time_var = xr.Variable('time', year_list)

# # Load in and concatenate all individual GeoTIFFs
# runoffs_allyears = xr.concat([rxr.open_rasterio(f"{url}{i}",chunks='auto') for i in fs.glob('/snowmelt/eric/mosaics/global_4326/*')],dim=time_var).squeeze().sortby('time')

# runoffs_allyears = runoffs_allyears.astype(np.int16)

# runoffs_allyears = runoffs_allyears.where(runoffs_allyears>-32768).astype(np.int16)

# runoffs_allyears_median = runoffs_allyears.median(dim='time')

# runoffs_allyears_median = runoffs_allyears_median.astype(np.int16).rio.write_nodata(-32768,encoded=True)

# with io.BytesIO() as buffer:
#         runoffs_allyears_median.rio.to_raster(buffer, driver="COG",dtype='int16')
#         buffer.seek(0)
#         blob_client = container_client.get_blob_client(f"snowmelt/eric/mosaics/global_4326/runoff_onset_mosaic_allyears_median_80m_4326.tif")
#         blob_client.upload_blob(buffer, overwrite=True)